**Set environment**

In [1]:
suppressMessages(suppressWarnings(source("../run_config_project_sing.R")))
suppressMessages(suppressWarnings(library(emmeans)))
suppressMessages(suppressWarnings(library(sandwich)))
show_env()

You are working on        Singularity: singularity_proj_encode_fcc 
BASE DIRECTORY (FD_BASE): /data/reddylab/Kuei 
REPO DIRECTORY (FD_REPO): /data/reddylab/Kuei/repo 
WORK DIRECTORY (FD_WORK): /data/reddylab/Kuei/work 
DATA DIRECTORY (FD_DATA): /data/reddylab/Kuei/data 

You are working with      ENCODE FCC 
PATH OF PROJECT (FD_PRJ): /data/reddylab/Kuei/repo/Proj_ENCODE_FCC 
PROJECT RESULTS (FD_RES): /data/reddylab/Kuei/repo/Proj_ENCODE_FCC/results 
PROJECT SCRIPTS (FD_EXE): /data/reddylab/Kuei/repo/Proj_ENCODE_FCC/scripts 
PROJECT DATA    (FD_DAT): /data/reddylab/Kuei/repo/Proj_ENCODE_FCC/data 
PROJECT NOTE    (FD_NBK): /data/reddylab/Kuei/repo/Proj_ENCODE_FCC/notebooks 
PROJECT DOCS    (FD_DOC): /data/reddylab/Kuei/repo/Proj_ENCODE_FCC/docs 
PROJECT LOG     (FD_LOG): /data/reddylab/Kuei/repo/Proj_ENCODE_FCC/log 
PROJECT REF     (FD_REF): /data/reddylab/Kuei/repo/Proj_ENCODE_FCC/references 



**Set global variable**

In [2]:
## init: region folder
vec = c("fcc_astarr_macs_input_overlap", "fcc_astarr_macs_input_union")
names(vec) = vec

VEC_TXT_FOLDER = vec
for (txt in vec) {cat(txt, "\n")}

fcc_astarr_macs_input_overlap 
fcc_astarr_macs_input_union 


In [3]:
vec = c("CRISPRi-HCRFF", "CRISPRi-Growth", "ENCODE-rE2G Benchmark", "CRISPR")
names(vec) = vec
VEC_TXT_ASSAY_OUT = vec

vec = c("ASTARR", "WSTARR", "LMPRA", "TMPRA", "ATAC")
names(vec) = vec
VEC_TXT_ASSAY_INP = vec

**Check input files**

In [4]:
txt_folder = "fcc_astarr_macs_input_union"
txt_fdiry  = file.path(FD_RES, "analysis_fcc_correlation", txt_folder)
txt_fname  = "*.rds"
txt_fglob  = file.path(txt_fdiry, txt_fname)

vec = Sys.glob(txt_fglob)
for (txt in vec) {cat(txt, "\n")}

/data/reddylab/Kuei/repo/Proj_ENCODE_FCC/results/analysis_fcc_correlation/fcc_astarr_macs_input_union/data.region.merge.group_crispr.score_starrmpra.rds 


## Import data

In [5]:
txt_foldr = "fcc_astarr_macs_input_union"
txt_fdiry = file.path(FD_RES, "analysis_fcc_correlation", txt_foldr)
txt_fname = "data.region.merge.group_crispr.score_starrmpra.rds"
txt_fpath = file.path(txt_fdiry, txt_fname)

lst = readRDS(txt_fpath)

lst_dat_region_merge = lst

res = lapply(lst, dim)
print(res)

dat = lst[[1]]
fun_display_table(head(dat, 3))

$`0.01`
[1] 610756      7

$`0.02`
[1] 610756      7

$`0.03`
[1] 610756      7

$`0.04`
[1] 610756      7

$`0.05`
[1] 610756      7

$`0.06`
[1] 610756      7

$`0.07`
[1] 610756      7

$`0.08`
[1] 610756      7

$`0.09`
[1] 610756      7

$`0.10`
[1] 610756      7

$`0.20`
[1] 610756      7

$`0.30`
[1] 610756      7

$`0.40`
[1] 610756      7

$`0.50`
[1] 610756      7



CutOff_CRISPR,Region,Assay_CRISPR,Group_CRISPR,Score_CRISPR,Assay_STARRMPRA,Score_STARRMPRA
0.01,chr10:100009096-100010466,CRISPR,0,-0.0378617,ASTARR,-1.2216566
0.01,chr10:100009096-100010466,CRISPR,0,-0.0378617,ATAC,2.3214952
0.01,chr10:100009096-100010466,CRISPR,0,-0.0378617,WSTARR,0.5896695


## Logit analysis: CRISPR ~ (STARRMPRA, ATAC)

In [6]:
### init
lst = lst_dat_region_merge

### loop through each threshold
lst = lapply(lst, function(dat_region_merge){
    ### loop through each CRISPR assay
    lst = lapply(VEC_TXT_ASSAY_OUT, function(txt_assay_out){
        ### arrange: get chosen CRISPR assay
        dat = dat_region_merge
        dat = dat %>%
            dplyr::filter(Assay_CRISPR == txt_assay_out) %>%
            dplyr::select(Region, Group_CRISPR, Assay_STARRMPRA, Score_STARRMPRA)
        
        ### arrange: pool STARR/MPRA together
        dat = dat %>% 
            dplyr::mutate(
                Assay_STARRMPRA = ifelse(Assay_STARRMPRA == "ATAC", "ATAC", "STARRMPRA")
            )

        ### arrange: set factor for each variable
        dat = dat %>%
            dplyr::mutate(
                Group_CRISPR    = factor(Group_CRISPR, levels = c(0, 1), labels = c("non-hit", "hit")),
                Assay_STARRMPRA = factor( Assay_STARRMPRA, levels = c("ATAC", "STARRMPRA")),
                Assay_STARRMPRA = relevel(Assay_STARRMPRA, ref = "ATAC")
            ) %>%
            dplyr::distinct()

        ### get assay count
        tab = table(dat$Assay_STARRMPRA)
        tab = as.data.frame(tab)
        colnames(tab) = c("Assay_STARRMPRA", "Count")
        
        ### apply logistic regression
        fit = glm(
            Group_CRISPR ~ Score_STARRMPRA * Assay_STARRMPRA, 
            data   = dat,
            family = binomial(link = "logit")
        )

        ### calculate coefficients (log-odds scale) and their CI
        V   = sandwich::vcovCL(fit, cluster = fit$model$Region) ### cluster-robust vcov
        res = emtrends(fit, specs = ~ Assay_STARRMPRA, var = "Score_STARRMPRA", vcov. = V)
        #res = emtrends(fit, specs = ~ Assay_STARRMPRA, var = "Score_STARRMPRA")
        
        ### get coefficient table
        dat = summary(res, infer = TRUE) |> as.data.frame()
        
        ### identify columns robustly across emmeans versions
        txt_cname_trend    = grep("\\.trend$", names(dat), value = TRUE)
        txt_cname_ci_lower = "asymp.LCL"
        txt_cname_ci_upper = "asymp.UCL"
        txt_cname_pval     = "p.value"
        
        ### summarize results
        out = dat %>%
            dplyr::transmute(
                Assay_STARRMPRA,
                OR               = exp(.data[[txt_cname_trend]]),
                OR_CI_Lower      = exp(.data[[txt_cname_ci_lower]]),
                OR_CI_Upper      = exp(.data[[txt_cname_ci_upper]]),
                Log10OR          = .data[[txt_cname_trend]]    / log(10),
                Log10OR_CI_Lower = .data[[txt_cname_ci_lower]] / log(10),
                Log10OR_CI_Upper = .data[[txt_cname_ci_upper]] / log(10),
                Pvalue           = .data[[txt_cname_pval]],
                Total            = nrow(fit$model)
            ) %>% 
            dplyr::left_join(tab, by = "Assay_STARRMPRA")
        return(out)
    })

    ### combine results across different input assay
    dat = bind_rows(lst, .id = "Assay_CRISPR")
    return(dat)
})

### concat results from different threshold
dat = bind_rows(lst, .id = "CutOff_CRISPR")
dat = dat %>% dplyr::mutate(CutOff_CRISPR = as.numeric(CutOff_CRISPR))

### assign and show
dat_logit_stats_pool = dat
print(dim(dat))
fun_display_table(head(dat))

[1] 112  12


CutOff_CRISPR,Assay_CRISPR,Assay_STARRMPRA,OR,OR_CI_Lower,OR_CI_Upper,Log10OR,Log10OR_CI_Lower,Log10OR_CI_Upper,Pvalue,Total,Count
0.01,CRISPRi-HCRFF,ATAC,2.378547,1.747075,3.238262,0.3763118,0.2423115,0.5103121,0e+00,5319,1330
0.01,CRISPRi-HCRFF,STARRMPRA,1.817318,1.490459,2.215858,0.2594310,0.1733200,0.3455420,0e+00,5319,3989
0.01,CRISPRi-Growth,ATAC,2.040152,1.946319,2.138509,0.3096625,0.2892139,0.3301110,0e+00,294769,80288
0.01,CRISPRi-Growth,STARRMPRA,1.801120,1.748739,1.855069,0.2555426,0.2427251,0.2683600,0e+00,294769,214481
0.01,ENCODE-rE2G Benchmark,ATAC,2.287777,1.662595,3.148044,0.3594136,0.2207865,0.4980408,4e-07,12400,3169
0.01,ENCODE-rE2G Benchmark,STARRMPRA,1.719379,1.532714,1.928777,0.2353716,0.1854612,0.2852819,0e+00,12400,9231


## Logit analysis: CRISPR ~ (STARR/MPRA, ATAC)

In [7]:
### init
lst = lst_dat_region_merge

### loop through each threshold
lst = lapply(lst, function(dat_region_merge){
    ### loop through each CRISPR assay
    lst = lapply(VEC_TXT_ASSAY_OUT, function(txt_assay_out){
        ### arrange: get chosen CRISPR assay
        dat = dat_region_merge
        dat = dat %>%
            dplyr::filter(Assay_CRISPR == txt_assay_out) %>%
            dplyr::select(Region, Group_CRISPR, Assay_STARRMPRA, Score_STARRMPRA)

        ### arrange: set factor for each variable
        dat = dat %>%
            dplyr::mutate(
                Group_CRISPR    = factor(Group_CRISPR, levels = c(0, 1), labels = c("non-hit", "hit")),
                Assay_STARRMPRA = factor( Assay_STARRMPRA, levels = VEC_TXT_ASSAY_INP),
                Assay_STARRMPRA = relevel(Assay_STARRMPRA, ref = "ATAC")
            ) %>%
            dplyr::distinct()

        ### get assay count
        tab = table(dat$Assay_STARRMPRA)
        tab = as.data.frame(tab)
        colnames(tab) = c("Assay_STARRMPRA", "Count")
        
        ### apply logistic regression
        fit = glm(
            Group_CRISPR ~ Score_STARRMPRA * Assay_STARRMPRA, 
            data   = dat,
            family = binomial(link = "logit")
        )

        ### calculate coefficients (log-odds scale) and their CI
        V   = sandwich::vcovCL(fit, cluster = fit$model$Region) ### cluster-robust vcov
        res = emtrends(fit, specs = ~ Assay_STARRMPRA, var = "Score_STARRMPRA", vcov. = V)
        #res = emtrends(fit, specs = ~ Assay_STARRMPRA, var = "Score_STARRMPRA")
        
        ### get coefficient table
        dat = summary(res, infer = TRUE) |> as.data.frame()
        
        ### identify columns robustly across emmeans versions
        txt_cname_trend    = grep("\\.trend$", names(dat), value = TRUE)
        txt_cname_ci_lower = "asymp.LCL"
        txt_cname_ci_upper = "asymp.UCL"
        txt_cname_pval     = "p.value"
        
        ### summarize results
        out = dat %>%
            dplyr::transmute(
                Assay_STARRMPRA,
                OR               = exp(.data[[txt_cname_trend]]),
                OR_CI_Lower      = exp(.data[[txt_cname_ci_lower]]),
                OR_CI_Upper      = exp(.data[[txt_cname_ci_upper]]),
                Log10OR          = .data[[txt_cname_trend]]    / log(10),
                Log10OR_CI_Lower = .data[[txt_cname_ci_lower]] / log(10),
                Log10OR_CI_Upper = .data[[txt_cname_ci_upper]] / log(10),
                Pvalue           = .data[[txt_cname_pval]],
                Total            = nrow(fit$model)
            ) %>% 
            dplyr::left_join(tab, by = "Assay_STARRMPRA")
        return(out)
    })

    ### combine results across different input assay
    dat = bind_rows(lst, .id = "Assay_CRISPR")
    return(dat)
})

### concat results from different threshold
dat = bind_rows(lst, .id = "CutOff_CRISPR")
dat = dat %>% dplyr::mutate(CutOff_CRISPR = as.numeric(CutOff_CRISPR))

### assign and show
dat_logit_stats_split = dat
print(dim(dat))
fun_display_table(head(dat))

[1] 280  12


CutOff_CRISPR,Assay_CRISPR,Assay_STARRMPRA,OR,OR_CI_Lower,OR_CI_Upper,Log10OR,Log10OR_CI_Lower,Log10OR_CI_Upper,Pvalue,Total,Count
0.01,CRISPRi-HCRFF,ATAC,2.378547,1.747075,3.238262,0.3763118,0.2423115,0.5103121,0.0000000,5319,1330
0.01,CRISPRi-HCRFF,ASTARR,2.065837,1.475816,2.891744,0.3150961,0.1690322,0.4611599,0.0000236,5319,1330
0.01,CRISPRi-HCRFF,WSTARR,1.860945,1.363876,2.539171,0.2697334,0.1347750,0.4046919,0.0000896,5319,1306
0.01,CRISPRi-HCRFF,LMPRA,1.400591,0.917296,2.138518,0.1463112,-0.0374905,0.3301129,0.1187161,5319,721
0.01,CRISPRi-HCRFF,TMPRA,2.393007,1.540039,3.718399,0.3789439,0.1875318,0.5703560,0.0001044,5319,632
0.01,CRISPRi-Growth,ATAC,2.040152,1.946319,2.138509,0.3096625,0.2892139,0.3301110,0.0000000,294769,80288


## Export results

In [8]:
### set file directory
txt_foldr = "fcc_astarr_macs_input_union"
txt_fdiry = file.path(FD_RES, "analysis_fcc_correlation", txt_foldr)
txt_fname = "result.cripsr_vs_starrmpra.logit.stats.pool.tsv"
txt_fpath = file.path(txt_fdiry, txt_fname)

### write table
dat = dat_logit_stats_pool
write_tsv(dat, txt_fpath)

In [9]:
### set file directory
txt_foldr = "fcc_astarr_macs_input_union"
txt_fdiry = file.path(FD_RES, "analysis_fcc_correlation", txt_foldr)
txt_fname = "result.cripsr_vs_starrmpra.logit.stats.split.tsv"
txt_fpath = file.path(txt_fdiry, txt_fname)

### write table
dat = dat_logit_stats_split
write_tsv(dat, txt_fpath)